In [ ]:
import lxml.html
import httpx 
import pandas as pd
import re

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_Chicago_alderpersons_since_1923"

In [ ]:
x = httpx.get(url)

In [ ]:
root = lxml.html.fromstring(x.text)

My plan here is to get all the ul items. And with that I need to get the sibling which is a div containing a title with the name of the ward

In [100]:
def compile_aldermen_for_wards(ul_element, ward_number, aldermen_dict):
    """
    Gather each the ward aldermen for wards whose aldermen are listed as bullet oints. 
    
    """
    ward_aldermen = []
    bullet_points = ul_element.cssselect("li")
    for bullet in bullet_points:
        links = bullet.cssselect('a')
        profile_link = None
        
        # I need to ensure I'm not counting people who are listed as images
        
        if links is not None:
            for element in links:
                href = element.get('href')
                if re.search("File", str(href), re.IGNORECASE) or \
                    re.search("cite_note", str(href), re.IGNORECASE):
                    continue
                else:
                    profile_link = "https://en.wikipedia.org/" + href
                    break
        # Now I should create a dictionary with the indivudual as the key and the links as the value
        # I will then add this dictionary to a list. This list will then be the value of final dictionary where the key is the ward
        person_dict = {}
        person_dict[bullet.text_content()] = profile_link
        ward_aldermen.append(person_dict)
    
    aldermen_dict[ward_number] = ward_aldermen
    
    return aldermen_dict
    

In [102]:
aldermen_dict = {}

body_content = root.cssselect('div.mw-body-content')[0]
heading = body_content.cssselect('div.mw-heading.mw-heading3')
list_elements = body_content.cssselect('ul')

for ward_number in heading:
    print(ward_number, ward_number.text_content())
    num = re.findall("\\d+", ward_number.text_content())
    ward = num[0]
    #print(ward_number.text_content())
    a = ward_number.getnext()
    #print("a =", (a))
    b = a.getnext()
    #print("b =", b)
    c = b.getnext()
    #print("c =", c)
    d = c.getnext()
    #print("d =", d)
    e = d.getnext()
    #print("e =", e)
    
    for element in [a, b, c, d, e]:
        if re.search("table", str(element), re.IGNORECASE):
            break
        elif re.search("ul", str(element), re.IGNORECASE):
            print(element)
            aldermen_dict = \
                compile_aldermen_for_wards(element, ward, aldermen_dict)
            
            print("after ward", ward_number.text_content(), aldermen_dict)
            break
            # go through loop
        else:
            continue
            
    # okay, I'll need to do something where
    # if class is table, break
    # if class is style, div, or p, continue
    # if class is ul
    ## go through the elements

    



        
    

<Element div at 0x12588a940> 1st Ward[edit]
<Element div at 0x125a08fa0> 2nd Ward[edit]
<Element div at 0x1258ddfe0> 3rd Ward[edit]
<Element div at 0x1258de800> 4th Ward[edit]
<Element div at 0x1258dde50> 5th Ward[edit]
<Element ul at 0x106c942d0>
after ward 5th Ward[edit] {'5': [{'Charles S. Eaton (Republican[parties 3])': None}, {'Leonard J. Grossman': None}, {'Charles S. Eaton (Republican[parties 3])': None}, {'Irving J. Schreiber': None}, {'James J. Cusack Jr. (Democratic[parties 1])': None}, {'Paul Howard Douglas': 'https://en.wikipedia.org//wiki/Paul_Howard_Douglas'}, {'Bertram B. Moss': None}, {'Robert E. Merriam': 'https://en.wikipedia.org//wiki/Robert_E._Merriam'}, {'Leon Despres': 'https://en.wikipedia.org//wiki/Leon_Despres'}, {'Ross Lathrop': None}, {'Lawrence Bloom': None}, {'Barbara Holt': None}, {'Leslie Hairston': 'https://en.wikipedia.org//wiki/Leslie_Hairston'}]}
<Element div at 0x12658e1c0> 6th Ward[edit]
<Element ul at 0x122c74820>
after ward 6th Ward[edit] {'5': [{

In [ ]:
aldermen_dict = {}

body_content = root.cssselect('div.mw-body-content')[0]
list_elements = body_content.cssselect('ul')

for aldermen in list_elements:
    # first, I need to associate each alder with a ward. To do that, I'm going 
    # to find the heading associated with each group
    
    
    # can I say something like, if a ward heading class exist between this element 
    # and the last element we went through, recognize a new ward?
    
    
    ward_heading = aldermen.getprevious()
        # Ward Heading
    if ward_heading is not None:
        print(ward_heading)
            # If we have a heading object, see if it contains ward
        if re.search("ward", ward_heading.text_content(), re.IGNORECASE):
                # Now find all digits
            num = re.findall("\\d+", ward_heading.text_content())
            ward = num[0]
            ward_aldermen = []
            print("WARD:", ward)
            print()
            
    # second, I find the bullet points and any links within them
    
    bullet_points = aldermen.cssselect("li")
    for bullet in bullet_points:
        links = bullet.cssselect('a')
        profile_link = None
        
        # I need to ensure I'm not counting people who are listed as images
        
        if links is not None:
            for element in links:
                href = element.get('href')
                if re.search("File", str(href), re.IGNORECASE) or \
                    re.search("cite_note", str(href), re.IGNORECASE):
                    continue
                else:
                    profile_link = "https://en.wikipedia.org/" + href
                    break
        #print("A person and any links", bullet.text_content(), profile_link)
        # Now I should create a dictionary with the indivudual as the key and the links as the value
        # I will then add this dictionary to a list. This list will then be the value of final dictionary where the key is the ward
        person_dict = {}
        person_dict[bullet.text_content()] = profile_link
        ward_aldermen.append(person_dict)
    
    aldermen_dict[ward] = ward_aldermen
    
        
        

   



Okay heck we aren't collecting these ward keys:

8th ward
12th 
13th
15th
16th
45th
48

In [ ]:
aldermen_dict

In [ ]:
people_dict = {}

body_content = root.cssselect('div.mw-body-content')
for b in bd:
    li = b.cssselect('ul')
    
    for l in li:
        print()
        print("MEET BULLET POINT", l.text_content())
        
        # Step 1: Grab Ward Number
        h = l.getprevious()
        # Ward Heading
        if h is not None:
            # If we have a heading object, see if it contains ward
            if re.search("ward", h.text_content(), re.IGNORECASE):
                # Now find all digits
                num = re.findall("\d+", h.text_content())
                people_dict[num[0]] = []
                
        # Each person in ward!
        
        # START HERE
        ## We need to re-work this so we are getting all the text and links all at once
            ## Then we can easily look if text is associated with a link -- thus if a person has a wiki page
            ## string split text based on /n, and then for every line in there (with a person's name and link info if available)
            ## add subsequent dictionary with person name as key, link as value



        print()
        print("a person?", l.text_content())
        x = l.cssselect("a")
        if x is not None:
            for ex in x:
                print("link? to external page about them?", ex.get("href"))


In [ ]:
people_dict